The task is simple: find the best combination of pre-processing steps and predictive models with respect to an objective criterion. Logistically this can be problematic: a small example might involve three classification models, and two data preprocessing steps with two possible variations for each — overall 12 combinations. For each of these combinations we would like to perform a grid search of predefined hyperparameters on a fixed cross-validation dataset, computing performance metrics for each option (for example ROC AUC). Clearly this can become complicated quickly. On the other hand, many of these combinations share substeps, and re-running such shared steps amounts to a loss of compute time.

## 1. Defining Pipelines Steps and Grid Search Parameters

The researcher specifies the possible processing steps and the scikit objects involved, then Reskit expands these steps to each possible pipeline. Reskit represents these pipelines in a convenient pandas dataframe, so the researcher can directly visualize and manipulate the experiments.

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

from sklearn.model_selection import StratifiedKFold

from reskit.core import Pipeliner


# Feature selection and feature extraction step variants (1st step)
feature_engineering = [('VT', VarianceThreshold()),
                       ('PCA', PCA())]

# Preprocessing step variants (2nd step)
scalers = [('standard', StandardScaler()),
           ('minmax', MinMaxScaler())]

# Models (3rd step)
classifiers = [('LR', LogisticRegression()),
               ('SVC', SVC()),
               ('SGD', SGDClassifier())]

# Reskit needs to define steps in this manner
steps = [('feature_engineering', feature_engineering),
         ('scaler', scalers),
         ('classifier', classifiers)]

# Grid search parameters for our models
param_grid = {'LR': {'penalty': ['l1', 'l2']},
              'SVC': {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
              'SGD': {'penalty': ['elasticnet'],
                      'l1_ratio': [0.1, 0.2, 0.3]}}

# Quality metric that we want to optimize
scoring='roc_auc'

# Setting cross-validations
grid_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
eval_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

pipe = Pipeliner(steps, grid_cv=grid_cv, eval_cv=eval_cv, param_grid=param_grid)
pipe.plan_table

,feature_engineering,scaler,classifier
0,VT,standard,LR
1,VT,standard,SVC
2,VT,standard,SGD
3,VT,minmax,LR
4,VT,minmax,SVC
5,VT,minmax,SGD
6,PCA,standard,LR
7,PCA,standard,SVC
8,PCA,standard,SGD
9,PCA,minmax,LR


## 2. Forbidden combinations

In case you don't want to use minmax scaler with SVC, you can define banned combo:

In [2]:
banned_combos = [('minmax', 'SVC')]
pipe = Pipeliner(steps, grid_cv=grid_cv, eval_cv=eval_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,feature_engineering,scaler,classifier
0,VT,standard,LR
1,VT,standard,SVC
2,VT,standard,SGD
3,VT,minmax,LR
4,VT,minmax,SGD
5,PCA,standard,LR
6,PCA,standard,SVC
7,PCA,standard,SGD
8,PCA,minmax,LR
9,PCA,minmax,SGD


## 3. Launching Experiment

Reskit then runs each experiment and presents results which are provided to the user through a pandas dataframe. For each pipeline’s classifier, Reskit grid search on cross-validation to find the best classifier’s parameters and report metric mean and standard deviation for each tested pipeline (ROC AUC in this case).

In [3]:
from sklearn.datasets import make_classification


X, y = make_classification()
pipe.get_results(X, y, scoring=['roc_auc'])

Line: 1/10
Line: 2/10
Line: 3/10
Line: 4/10
Line: 5/10
Line: 6/10
Line: 7/10
Line: 8/10
Line: 9/10
Line: 10/10


,feature_engineering,scaler,classifier,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,VT,standard,LR,0.982,0.0222711,{'penalty': 'l1'},0.984,0.0233238,[ 0.98 1. 1. 0.94 1. ]
1,VT,standard,SVC,0.978,0.0299333,{'kernel': 'rbf'},0.984,0.0162481,[ 0.97 1. 0.99 0.96 1. ]
2,VT,standard,SGD,0.916,0.0574804,"{'l1_ratio': 0.2, 'penalty': 'elasticnet'}",0.936,0.0523832,[ 0.91 1. 0.99 0.86 0.92]
3,VT,minmax,LR,0.99,0.0126491,{'penalty': 'l1'},0.984,0.032,[ 1. 1. 1. 0.92 1. ]
4,VT,minmax,SGD,0.962,0.0285657,"{'l1_ratio': 0.2, 'penalty': 'elasticnet'}",0.978,0.016,[ 0.98 1. 0.96 0.96 0.99]
5,PCA,standard,LR,0.958,0.0305941,{'penalty': 'l2'},0.93,0.0579655,[ 0.88 1. 0.9 0.87 1. ]
6,PCA,standard,SVC,0.96,0.0260768,{'kernel': 'rbf'},0.894,0.0821219,[ 0.79 1. 0.91 0.81 0.96]
7,PCA,standard,SGD,0.92,0.0352136,"{'l1_ratio': 0.2, 'penalty': 'elasticnet'}",0.892,0.0570614,[ 0.84 0.98 0.85 0.85 0.94]
8,PCA,minmax,LR,0.966,0.0287054,{'penalty': 'l2'},0.944,0.0488262,[ 1. 0.99 0.95 0.87 0.91]
9,PCA,minmax,SGD,0.958,0.0285657,"{'l1_ratio': 0.3, 'penalty': 'elasticnet'}",0.914,0.0422374,[ 0.92 0.99 0.88 0.87 0.91]


## 4. Caching intermediate steps

Reskit also allows you to cache interim calculations to avoid unnecessary recalculations.

In [4]:
from sklearn.preprocessing import Binarizer

# Simple binarization step that we want ot cache
binarizer = [('binarizer', Binarizer())]

# Reskit needs to define steps in this manner
steps = [('binarizer', binarizer),
         ('classifier', classifiers)]

pipe = Pipeliner(steps, grid_cv=grid_cv, eval_cv=eval_cv, param_grid=param_grid)
pipe.plan_table

,binarizer,classifier
0,binarizer,LR
1,binarizer,SVC
2,binarizer,SGD


In [5]:
pipe.get_results(X, y, caching_steps=['binarizer'])

Line: 1/3
Line: 2/3
Line: 3/3


,binarizer,classifier,grid_accuracy_mean,grid_accuracy_std,grid_accuracy_best_params,eval_accuracy_mean,eval_accuracy_std,eval_accuracy_scores
0,binarizer,LR,0.98,0.0244949,{'penalty': 'l1'},0.98,0.04,[ 1. 1. 1. 0.9 1. ]
1,binarizer,SVC,0.98,0.0244949,{'kernel': 'linear'},0.98,0.04,[ 1. 1. 1. 0.9 1. ]
2,binarizer,SGD,0.92,0.0244949,"{'l1_ratio': 0.3, 'penalty': 'elasticnet'}",0.92,0.0927362,[ 1. 0.75 1. 0.9 0.95]


Last cached calculations stored in _cached_X

In [6]:
pipe._cached_X

OrderedDict([('init',
              array([[  1.09937530e+00,   1.02915387e+00,  -2.97047637e-01, ...,
                        1.37549721e+00,   1.36015788e+00,   6.38291714e-02],
                     [  1.93034431e-01,   2.85761376e-01,  -2.18289189e-01, ...,
                        1.41887760e+00,   1.65096951e+00,   8.19783664e-01],
                     [  1.23930535e+00,  -1.41600902e+00,  -7.30964779e-01, ...,
                        9.74122429e-01,  -3.36947062e-01,   3.49815771e-01],
                     ..., 
                     [  1.73037297e+00,  -1.21157917e-01,  -9.51976123e-04, ...,
                        6.15791285e-01,   1.73043934e+00,   2.86563008e-01],
                     [  4.47412837e-01,  -5.44357284e-02,   1.27346136e+00, ...,
                       -7.74333429e-01,   1.28615892e+00,  -9.54820144e-02],
                     [ -3.60479519e-02,  -5.05281208e-01,   5.27193469e-02, ...,
                        6.84841594e-01,   5.78002550e-01,   1.72943997e-01]])),
